**Table of contents**<a id='toc0_'></a>    
- [Ollama](#toc1_)      
- [Agents](#toc3_)    
  - [UserProxyAgent](#toc3_1_)    
  - [ConversableAgent](#toc3_2_)    
  - [AssistantAgent](#toc3_3_)    
  - [GroupChatManager](#toc3_4_)    
  - [LocalCommandLineCodeExecutor](#toc3_5_)    
    - [code_execution_config](#toc3_5_1_)    
- [Implementing Custom Agents](#toc4_)    
  - [RAG Agent](#toc4_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Ollama](#toc0_)

# <a id='toc2_'></a>[](#toc0_)
We want to use AutoGen alongside Ollama. To do this, we have to follow some steps first.
Since AutoGent is integrated with the OpenAI API, we have to find a workaround to be able to use Ollama with it.
A solution, is Launching our local LLM into a local API.

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1', # Local ollama address
    api_key='ollama', # required, but unused
)

response = client.chat.completions.create(
  model="llama3.1",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The LA Dodgers won in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

It was actually played at Globe Life Field in Arlington, Texas (hosting team Dallas Rangers) and other various AL and NL parks due to Covid-19 protocols allowing teams to have a home game designation. However the final 6 games were hosted at Globe life field. The Dodgers played the Tampa Bay Rays, but only had to play the minimum 4 games necessary for a series win in the World Series that year


Now that everything is working, we can start importing our agents and designing our system using AutoGen. First of all, we will need an llm config for each agent. we can customize them for each agent specigically, or use one config for all of them.

In [ ]:
from autogen import AssistantAgent, UserProxyAgent

config_list = [
  {
    "model": "llama3.1",
    "base_url": "http://localhost:11434/v1",
    "api_key": "ollama",
  }
]

llm_config = {
    "config_list": config_list,
    "seed": 42,
}

# <a id='toc3_'></a>[Agents](#toc0_)

## <a id='toc3_1_'></a>[UserProxyAgent](#toc0_)
The user proxy agent is one of the basic agents. It represents a human user in the conversation. In simple terms, this agent is responsible for carrying your commands to the rest of the agents.

In [ ]:
from autogen import UserProxyAgent

user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS", 
    max_consecutive_auto_reply=10, 
)

## <a id='toc3_2_'></a>[ConversableAgent](#toc0_)
This is the simplest of agents. It is the base class for agents capable of conversation.

In [ ]:
from autogen import ConversableAgent

conversable_agent = ConversableAgent(
    name="conversable_agent",
    llm_config=llm_config,
)

reply = conversable_agent.generate_reply(
    messages=[{"role": "user", "content": "Hello, who are you?"}],
    sender=user_proxy
)
print(reply)

## <a id='toc3_3_'></a>[AssistantAgent](#toc0_)
This AI Agent is one of the key agents in the system. it can assist with various tasks such as general QA, writing, problem solving, code generation, code explanation, data analysis, task planning, translation, maths, researching etc.

In [ ]:
assistant = AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

reply = assistant.generate_reply(
    messages=[{"role": "user", "content": "Hello, who are you?"}],
    sender=user_proxy
)
print(reply)

## <a id='toc3_4_'></a>[GroupChatManager](#toc0_)
You can think of this agent as the manager of a work group. The manager, as the name suggests, manages the conversations between multiple agents.

In [ ]:
from autogen import GroupChatManager, GroupChat

# Create multiple agents
agent1 = AssistantAgent("agent1", llm_config=llm_config)
agent2 = AssistantAgent("agent2", llm_config=llm_config)

# Create a GroupChatManager
group_chat = GroupChat(
    agents=[user_proxy, assistant, agent1, agent2],
    messages=[],
    max_round=12
)

# Create a GroupChatManager
manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)

user_proxy.initiate_chat(
    manager,
    message="Discuss the pros and cons of renewable energy."
)

## <a id='toc3_5_'></a>[LocalCommandLineCodeExecutor](#toc0_)
This agent is probably the most advanced of the agents thus far. Not only can it generate code, but it can also execute the code it writes, given the right environments.

### <a id='toc3_5_1_'></a>[code_execution_config](#toc0_)
The code execution config defines the parameters for code execution, including which executor to use, where to execute the code, and any specific settings for the execution environment.

The parameters for this config include:
1- executor
2- work_dir
3- use_docker
4- last_n_messages

In [ ]:
from autogen.coding import LocalCommandLineCodeExecutor
import os

executor = LocalCommandLineCodeExecutor(
    timeout=60,  # timeout in seconds
    work_dir="coding"  # working directory
)

code_execution_config = {
    "executor": executor,
    "execution_policies": {
        "python": True,  # Allow execution of Python code
        "shell": True  # False: Only save shell scripts, do not execute/ True: executes the script as well
    },
    "last_n_messages": 3,
}

code_assistant = ConversableAgent(
    name="code_executor_agent",
    llm_config=llm_config,
    code_execution_config=code_execution_config
)

response = user_proxy.initiate_chat(
    code_assistant,
    message="write a code to calculate the sin of x. store it in your working directory. run the code to make sure it is correct. if you encounter an error or the answer is wrong, write the code again."
)
print(response)